https://api.together.ai/

#Liste des imports

##Modules généraux

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import subprocess
import shutil
import time
import ipywidgets as widgets
from IPython.display import display, Audio, clear_output
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import re
import random
import sys
# from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip
import cv2
import os
import torch
from PIL import Image, ImageDraw, ImageFont


In [ ]:
try:
    import together
    try:
        version = together.__version__
        print(f"Together est déjà installé, version {version}")
    except AttributeError:
        print("Together est déjà installé, version inconnue")
except ImportError:
    print("Together n'est pas installé. Installation en cours...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "together"])
    import together
    print("Together installé avec succès")


Together est déjà installé, version inconnue


##Modules IA

In [ ]:
# Vérifier si 'whisper' est installé
try:
    import whisper
    print(f"Whisper est déjà installé, version {whisper.__version__}")
except ImportError:
    print("Whisper n'est pas installé. Installation en cours...")
    !pip install openai-whisper
    import whisper
    print(f"Whisper installé, version {whisper.__version__}")

from PIL import Image
!pip install --upgrade transformers
import transformers
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer ,AutoTokenizer, AutoModelForSeq2SeqLM
import moviepy.editor as mp
!pip install python-bidi




Whisper est déjà installé, version 20250625


Exception ignored in: <function _xla_gc_callback at 0x7ae7fa2399e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


#Structure générale du projet

##Fonctions et variables globales du code

###Variables globales du code

####Colab

In [ ]:
# Chemins généraux
chemin_script_général = os.path.split(os.getcwd())[0]
chemin_modeles = os.path.join(chemin_script_général, "Modèles et polices")
chemin_polices = os.path.join(chemin_modeles, "Polices")
police_path = os.path.join(chemin_script_général,"Modèles et polices/Polices/Rubik/static/Rubik-Regular.ttf"
logo_path = os.path.join(chemin_script_général,"Logo.png")
follow_path = os.path.join(chemin_script_général,"Réseaux à suivre.jpg")
modele_textuel_choisi = 'small'
# Chemins des dossiers vidéo
chemin_dossier_0 = os.path.join(chemin_script_général, "0.Vidéos sans sous dossier")
chemin_dossier_1 = os.path.join(chemin_script_général, "1.Vidéos vierges avec dossier")
chemin_dossier_2 = os.path.join(chemin_script_général, "2.Sous-titres Extrait")
chemin_dossier_3 = os.path.join(chemin_script_général, "3.Sous-titres Traduit (modèle)")
chemin_dossier_4 = os.path.join(chemin_script_général, "4.Sous-titres Vérifié")
chemin_dossier_5 = os.path.join(chemin_script_général, "5.Vidéos Montées")

extension_transcription_avec_timecode = "_transcription_avec_timecodes.txt"
extension_transcription_sans_timecode = "_transcription_sans_timecodes.txt"
extension_traduit = "_traduit.txt"
extension_revise = "_revise.txt"
extension_titre = "_titre.txt"
extension_montee = "_final.mp4"

# Chemin de sortie
chemin_dossier_sortie = chemin_dossier_5


# Titre final
title_duration = 0.05
message_abonnement_duration = 2

###Fonctions globales du code

####Manipuler dossiers

#####Listing de dossiers ou éléments

In [ ]:
def lister_elements_videos_dossier(dossier, extensions_videos=None, chemin_complet=True):
    """
    Liste tous les fichiers vidéo dans un dossier donné.

    :param dossier: Chemin du dossier à analyser.
    :param extensions_videos: Liste des extensions vidéo à considérer (par défaut : extensions courantes).
    :param chemin_complet: Indique si on retourne le chemin complet ou seulement le nom du fichier.
    :return: Liste des chemins ou des noms des fichiers vidéo trouvés.
    """
    # Extensions vidéo par défaut si non spécifiées
    if extensions_videos is None:
        extensions_videos = ['.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.mpeg', '.mpg', '.3gp','.m4a']

    videos = []

    # Parcourir tous les fichiers du dossier
    try:
        for element in os.listdir(dossier):
            chemin_complet_element = os.path.join(dossier, element)
            if os.path.isfile(chemin_complet_element) and os.path.splitext(element)[-1].lower() in extensions_videos:
                if chemin_complet:
                    videos.append(chemin_complet_element)
                else:
                    videos.append(element)
    except FileNotFoundError:
        print("Le dossier spécifié n'existe pas.")
    except PermissionError:
        print("Accès refusé au dossier spécifié.")

    return videos



# # Exemple d'utilisation
# videos = lister_elements_videos_dossier(chemin_dossier_entree, chemin_complet=False)
# print("Vidéos trouvées sans path:", videos)


# # Exemple d'utilisation
# videos = lister_elements_videos_dossier(chemin_dossier_entree, chemin_complet=True)
# print("Vidéos trouvées avec path:", videos)


In [ ]:
def lister_videos_dans_sous_dossiers_uniquement(dossier, extensions_videos=None, chemin_complet=True):
    """
    Liste tous les fichiers vidéo présents uniquement dans les sous-dossiers.

    :param dossier: Chemin du dossier à analyser.
    :param extensions_videos: Liste des extensions vidéo à considérer (par défaut : extensions courantes).
    :param chemin_complet: Indique si on retourne le chemin complet ou seulement le nom du fichier.
    :return: Liste des chemins ou des noms des fichiers vidéo trouvés dans les sous-dossiers.
    """
    # Extensions vidéo par défaut si non spécifiées
    if extensions_videos is None:
        extensions_videos = ['.m4a','.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.mpeg', '.mpg', '.3gp']

    videos = []

    # Parcourir tous les fichiers et sous-dossiers
    try:
        for root, _, files in os.walk(dossier):
            # Ignorer le dossier racine, ne traiter que les sous-dossiers
            if root != dossier:
                for file in files:
                    if os.path.splitext(file)[-1].lower() in extensions_videos:
                        if chemin_complet:
                            videos.append(os.path.join(root, file))
                        else:
                            videos.append(file)
    except FileNotFoundError:
        print("Le dossier spécifié n'existe pas.")
    except PermissionError:
        print("Accès refusé au dossier spécifié.")

    return videos

# # Appel de la fonction pour obtenir les noms de fichiers uniquement
# videos_noms = lister_videos_dans_sous_dossiers_uniquement(chemin_dossier_entree, chemin_complet=False)
# print("Noms des vidéos trouvées dans les sous-dossiers :", videos_noms)

# # Appel de la fonction pour obtenir les chemins complets
# videos_chemins = lister_videos_dans_sous_dossiers_uniquement(chemin_dossier_entree, chemin_complet=True)
# print("Chemins complets des vidéos trouvées dans les sous-dossiers :", videos_chemins)


In [ ]:
def lister_sous_dossiers_avec_videos(dossier, extensions_videos=None, chemin_complet=True):
    """
    Liste tous les sous-dossiers contenant des fichiers vidéo.

    :param dossier: Chemin du dossier à analyser.
    :param extensions_videos: Liste des extensions vidéo à considérer (par défaut : extensions courantes).
    :param chemin_complet: Indique si on retourne le chemin complet ou seulement le nom des sous-dossiers.
    :return: Liste des sous-dossiers contenant des vidéos.
    """
    # Extensions vidéo par défaut si non spécifiées
    if extensions_videos is None:
        extensions_videos = ['.m4a','.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm', '.mpeg', '.mpg', '.3gp']

    sous_dossiers_avec_videos = []

    # Vérification si le dossier existe
    if not os.path.exists(dossier):
        print("Le dossier spécifié n'existe pas.")
        return []

    try:
        # Parcourir les sous-dossiers directs du dossier
        for entry in os.scandir(dossier):
            if entry.is_dir():  # Vérifie si l'entrée est un sous-dossier
                # Parcourir les fichiers dans le sous-dossier
                for file in os.listdir(entry.path):
                    if os.path.splitext(file)[-1].lower() in extensions_videos:
                        if chemin_complet:
                            sous_dossiers_avec_videos.append(entry.path)  # Ajouter le chemin complet du sous-dossier
                        else:
                            sous_dossiers_avec_videos.append(entry.name)  # Ajouter juste le nom du sous-dossier
                        break  # Arrêter dès qu'une vidéo est trouvée dans ce sous-dossier
    except PermissionError:
        print("Accès refusé au dossier spécifié.")
    except Exception as e:
        print(f"Erreur lors de la lecture du dossier : {e}")

    return sous_dossiers_avec_videos

# # Exemple d'utilisation
# sous_dossiers_avec_videos = lister_sous_dossiers_avec_videos(chemin_dossier, chemin_complet=True)
# print("Sous-dossiers avec des vidéos :", sous_dossiers_avec_videos)


In [ ]:
def trouver_audio_dans_dossier(chemin_dossier):
    """
    Recherche un fichier audio dans un dossier donné.

    Args:
        chemin_dossier (str): Le chemin vers le dossier à analyser.

    Returns:
        str: Le chemin complet du fichier audio trouvé, ou None si aucun fichier audio n'est trouvé.
    """
    extensions_audio = {".mp3", ".wav", ".flac", ".aac", ".ogg", ".m4a"}  # Ajouter les extensions nécessaires

    # Parcourir les fichiers dans le dossier
    for fichier in os.listdir(chemin_dossier):
        chemin_fichier = os.path.join(chemin_dossier, fichier)
        # Vérifier si c'est un fichier et s'il a une extension audio
        if os.path.isfile(chemin_fichier) and os.path.splitext(fichier)[-1].lower() in extensions_audio:
            return chemin_fichier

    return None


In [ ]:
def recuperer_fichiers_par_suffixe(dossier, suffixe):
    """
    Récupérer tous les fichiers d'un dossier qui se terminent par un certain suffixe.

    :param dossier: Chemin du dossier dans lequel chercher les fichiers.
    :param suffixe: Suffixe du fichier à chercher (par exemple "_transcription_avec_timecodes.txt").
    :return: Liste des chemins des fichiers correspondants.
    """
    fichiers_correspondants = []

    # Parcourir tous les fichiers du dossier
    for fichier in os.listdir(dossier):
        chemin_fichier = os.path.join(dossier, fichier)

        # Vérifier si c'est un fichier et si son nom se termine par le suffixe donné
        if os.path.isfile(chemin_fichier) and fichier.endswith(suffixe):
            fichiers_correspondants.append(chemin_fichier)

    return fichiers_correspondants

# # Exemple d'utilisation
# chemin_dossier = "/content/drive/Othercomputers/Mon ordinateur portable/Projet concret (Drive)/2.Sous-titres Extrait/بشرى فضل المشي الى صلاة الجمعة. عزيز فرحان العنزي"
# suffixe = "_transcription_avec_timecodes.txt"

# # Récupérer tous les fichiers avec le suffixe "_transcription_avec_timecodes.txt"
# fichiers = recuperer_fichiers_par_suffixe(chemin_dossier, suffixe)

# # Afficher les résultats
# print("Fichiers trouvés :")
# for fichier in fichiers:
#     print(fichier)


#####Générations de dossiers pour vidéos

In [ ]:
def generer_dossier_et_deplacer_video_dedans(video_path):
    """
    Crée un sous-dossier portant le nom de la vidéo (sans extension) et déplace la vidéo dedans.

    :param video_path: Chemin du fichier vidéo à déplacer.
    :return: True si l'opération réussit, False sinon.
    """
    # Vérifier si le fichier vidéo existe
    if not os.path.isfile(video_path):
        print("Le fichier vidéo n'existe pas.")
        return False

    try:
        # Extraction du nom de base du fichier vidéo (sans extension)
        video_name = os.path.splitext(os.path.basename(video_path))[0]

        # Création du sous-dossier avec le nom de la vidéo
        folder_path = os.path.join(os.path.dirname(video_path), video_name)

        # Si le dossier existe déjà, on ne le recrée pas
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        # Nouveau chemin pour la vidéo dans le sous-dossier
        new_video_path = os.path.join(folder_path, os.path.basename(video_path))

        # Déplacer la vidéo dans le sous-dossier
        shutil.move(video_path, new_video_path)
        print(f"La vidéo a été déplacée vers : {new_video_path}")
        return True

    except Exception as e:
        print(f"Erreur lors du déplacement de la vidéo : {e}")
        return False

# # Exemple d'utilisation
# if generer_dossier_et_deplacer_video_dedans(temp_video_path):
#     print("L'opération a été effectuée avec succès.")
# else:
#     print("L'opération a échoué.")


#####Déplacement de sous dossiers

In [ ]:
def deplacer_dossier(source, destination):
    """
    Déplace un dossier (et son contenu) de 'source' vers 'destination'.

    Args:
    - source : chemin du dossier à déplacer.
    - destination : chemin du dossier de destination.

    Retourne:
    - True si l'opération réussit, False sinon.
    """
    try:
        # Vérifier si le dossier source existe
        if not os.path.exists(source):
            print(f"Le dossier source '{source}' n'existe pas.")
            return False

        # Vérifier si le dossier de destination existe, sinon le créer
        if not os.path.exists(destination):
            os.makedirs(destination)

        # Déplacer le dossier et son contenu
        shutil.move(source, destination)
        print(f"Le dossier '{source}' a été déplacé vers '{destination}'.")
        return True
    except Exception as e:
        print(f"Une erreur est survenue : {e}")
        return False


#####Déplacer vidéos

In [ ]:
import shutil
import os

def deplacer_video(chemin_source, dossier_destination):
    """
    Déplace une vidéo depuis son emplacement source vers un dossier cible.

    Args:
        chemin_source (str): Chemin complet de la vidéo à déplacer.
        dossier_destination (str): Chemin du dossier de destination.
    """
    try:
        # Vérifier si le fichier source existe
        if not os.path.exists(chemin_source):
            print(f"Le fichier source n'existe pas : {chemin_source}")
            return

        # Vérifier si le dossier de destination existe, sinon le créer
        if not os.path.exists(dossier_destination):
            os.makedirs(dossier_destination)
            print(f"Dossier créé : {dossier_destination}")

        # Construire le chemin final pour la vidéo
        nom_fichier = os.path.basename(chemin_source)
        chemin_final = os.path.join(dossier_destination, nom_fichier)

        # Déplacer le fichier
        shutil.move(chemin_source, chemin_final)
        print(f"Vidéo déplacée avec succès vers : {chemin_final}")

    except Exception as e:
        print(f"Erreur lors du déplacement : {e}")


####Opérations d'extractions

#####Extraire audio

In [ ]:
def extraire_audio_de_video(chemin_video):
    """
    Extrait l'audio d'une vidéo et le sauvegarde dans le même dossier que la vidéo.
    Si le fichier audio existe déjà, il sera écrasé.

    :param chemin_video: Le chemin du fichier vidéo à partir duquel extraire l'audio.
    :return: Le chemin du fichier audio extrait ou None en cas d'erreur.
    """
    if not os.path.isfile(chemin_video):
        print(f"Le fichier vidéo n'existe pas : {chemin_video}")
        return None

    dossier_video = os.path.dirname(chemin_video)
    nom_fichier_video = os.path.splitext(os.path.basename(chemin_video))[0]
    chemin_audio = os.path.join(dossier_video, f"{nom_fichier_video}.mp3")

    commande = [
        "ffmpeg",
        "-y",  # Force l'écrasement des fichiers existants
        "-i", chemin_video,
        "-vn",
        "-acodec", "mp3",
        chemin_audio
    ]

    try:
        subprocess.run(commande, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"L'audio a été extrait avec succès : {chemin_audio}")
        return chemin_audio
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de l'extraction de l'audio : {e.stderr.decode('utf-8')}")
        return None


#####Extraire titre

In [ ]:
def extract_title_from_path(title: str):
    """
    Extrait le nom du fichier sans extension depuis un chemin de fichier ou un titre avec extension.

    Args:
        title (str): Le chemin du fichier ou titre avec ou sans extension.

    Returns:
        tuple: Titre sans extension, extension
    """
    # Si c'est un chemin de fichier, extraire le nom sans l'extension et l'extension
    file_name, file_extension = os.path.splitext(os.path.basename(title))
    return file_name, file_extension

# def translate_video_title(file_name, file_extension, tokenizer, model, input_lang="arb_Arab", target_lang="fra_Latn"):
#     """
#     Traduit le titre d'une vidéo d'une langue source vers une langue cible à l'aide du modèle de traduction.
#     Si le titre contient une extension vidéo, seule la partie titre est traduite, l'extension est conservée.

#     :param title: Le titre de la vidéo (sous forme de texte brut, avec ou sans extension).
#     :param input_lang: Langue de l'entrée (par exemple 'ar' pour arabe).
#     :param target_lang: Langue cible pour la traduction (par exemple 'fr' pour français).
#     :return: Le titre traduit avec son extension.
#     """
#     # Extraire le titre sans l'extension et l'extension du fichier
#     print("jojo1")
#     # Traduire uniquement le texte du titre (sans extension)
#     titre_traduit = traduire_texte_segment(file_name, tokenizer, model, input_lang, target_lang)

#     # Ajouter l'extension d'origine au titre traduit
#     translated_title_with_extension = titre_traduit + file_extension

#     return translated_title_with_extension



##Extraction du texte

###Extraction si sous titres présents dans la vidéo

####Fonctions

In [ ]:
def extraire_sous_titres_si_deja_inclus_dans_video_vlc(video_path, output_folder):
    """
    Extrait les sous-titres d'une vidéo à l'aide de ffmpeg et les sauvegarde dans un fichier SRT.
    Retourne un booléen pour indiquer le succès ou l'échec de l'opération.

    :param video_path: Chemin du fichier vidéo.
    :param output_folder: Dossier où sauvegarder les sous-titres extraits.
    :return: True si l'extraction réussit, False sinon.
    """
    # Vérifier si la vidéo existe
    if not os.path.isfile(video_path):
        print("Erreur : Le fichier vidéo spécifié n'existe pas.")
        return False

    # Assurez-vous que le dossier de sortie existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Extraction du nom de base du fichier vidéo
    video_name = os.path.basename(video_path)
    # Nom du fichier de sous-titres de sortie
    subtitles_path = os.path.join(output_folder, "subtitles.srt")

    # Commande ffmpeg pour extraire les sous-titres
    command = [
        'ffmpeg', '-i', video_path,
        '-map', '0:s:0',
        subtitles_path
    ]

    # Exécution de la commande ffmpeg
    try:
        subprocess.run(command, check=True, text=True, stderr=subprocess.PIPE)
        print(f"Les sous-titres ont été extraits avec succès et sauvegardés dans {subtitles_path}")
        return True
    except subprocess.CalledProcessError as e:
        print("Erreur lors de l'extraction des sous-titres.")
        print(f"Sortie de l'erreur : {e.stderr}")
        return False



###Génération si sous-titres absents (à privilégier très fortement)

####Charger modèle

In [ ]:
import os

def charger_modele_textuel(chemin_modeles, modele = modele_textuel_choisi):
    """
    Charge un modèle Whisper à partir d'un chemin spécifié.

    Args:
        chemin_modeles (str): Chemin du dossier contenant les modèles.
        modele (str): Nom du modèle (sans extension .pt).

    Returns:
        whisper.Model: Le modèle chargé si réussi, ou None en cas d'erreur.
    """
    chemin_modele_complet = os.path.join(chemin_modeles, f"{modele}.pt")

    # Vérification de l'existence du fichier modèle
    if not os.path.isfile(chemin_modele_complet):
        print(f"Le modèle '{modele}.pt' n'a pas été trouvé dans le chemin spécifié : {chemin_modele_complet}")
        return None

    # Tentative de chargement du modèle
    try:
        modele_whisper = whisper.load_model(chemin_modele_complet)
        print(f"Modèle '{modele}' chargé avec succès.")
        return modele_whisper
    except Exception as e:
        print(f"Erreur lors du chargement du modèle : {e}")
        return None


####Sans timecode

In [ ]:
def transcrire_audio_vers_texte_sans_code_temps(fichier_audio, modele_whisper, langue="ar"):
    """
    Transcrire les fichiers audio en texte sans code temporel en utilisant un modèle Whisper à partir d'un chemin spécifique.

    :param fichiers_audio: Liste des chemins vers les fichiers audio.
    :param chemin_modeles: Le chemin vers le dossier contenant les modèles Whisper.
    :param modele: Le modèle Whisper à utiliser (par défaut : "base").
    :param langue: La langue de l'audio (par défaut : "ar" pour arabe).
    :return: Liste des textes transcrits pour chaque fichier audio.
    """
    # Construire le chemin complet du modèle avec l'extension ".pt"

    transcriptions = []
    jojo = 0
    try:
        jojo += 1
        print("On est au fichier numéro =", jojo)
        print("Fichier audio = " , fichier_audio)
        # Transcrire le fichier audio
        resultat = modele_whisper.transcribe(fichier_audio, language=langue, fp16=False)

        # Extraire le texte sans le code temporel
        transcription = resultat["text"]
        transcriptions.append(transcription)

        # Sauvegarder la transcription dans un fichier texte dans le même dossier que l'audio
        dossier_audio = os.path.dirname(fichier_audio)
        nom_fichier_audio = os.path.splitext(os.path.basename(fichier_audio))[0]
        chemin_fichier_transcription = os.path.join(dossier_audio, f"{nom_fichier_audio}{extension_transcription_sans_timecode}")

        with open(chemin_fichier_transcription, "w", encoding="utf-8") as f:
            f.write(transcription)

        print(f"Transcription pour {fichier_audio} sauvegardée dans {chemin_fichier_transcription}")

    except Exception as e:
        print(f"Erreur lors de la transcription de {fichier_audio}: {e}")
        transcriptions.append(None)

    return transcriptions





####Avec timecode

In [ ]:
import os

def transcrire_audio_vers_texte_avec_code_temps(fichier_audio, modele_whisper, langue="ar", extension_transcription_avec_timecode=extension_transcription_avec_timecode):
    try:
        # Transcription avec des timecodes
        resultat = modele_whisper.transcribe(fichier_audio, language=langue, fp16=False)

        transcription = ""
        for segment in resultat["segments"]:
            debut = segment["start"]
            fin = segment["end"]
            texte = segment["text"]
            transcription += f"[{debut:.2f} - {fin:.2f}] {texte}\n"

        # Construire le chemin du fichier de transcription avec timecodes
        dossier_audio = os.path.dirname(fichier_audio)
        nom_fichier_audio = os.path.splitext(os.path.basename(fichier_audio))[0]
        chemin_fichier_transcription = os.path.join(dossier_audio, f"{nom_fichier_audio}{extension_transcription_avec_timecode}")

        # Sauvegarder la transcription avec timecodes dans un fichier
        with open(chemin_fichier_transcription, "w", encoding="utf-8") as f:
            f.write(transcription)

        print(f"Transcription avec timecodes pour {fichier_audio} sauvegardée dans {chemin_fichier_transcription}")
        return transcription  # Retourne la transcription complète

    except Exception as e:
        erreur_message = f"Erreur lors de la transcription de {fichier_audio}: {e}"
        print(erreur_message)
        return erreur_message  # Retourne le message d'erreur





####Tests

In [ ]:
import os
import time

def tester_modeles_et_afficher_resultats(fichiers_audio, chemin_modeles, langue="ar"):
    # Récupérer tous les modèles dans le dossier Models
    modeles_disponibles = [f for f in os.listdir(chemin_modeles) if f.endswith('.pt')]

    tableau_recapitulatif = []

    for modele_fichier in modeles_disponibles:
        modele = os.path.splitext(modele_fichier)[0]  # Extrait le nom du modèle sans l'extension '.pt'
        print(f"Test du modèle : {modele}")

        # Démarrer la chronométrie pour mesurer le temps de transcription
        start_time = time.time()

        # Transcrire l'audio avec le modèle actuel
        textes_transcrits = transcrire_audio_vers_texte_avec_code_temps(fichiers_audio, chemin_modeles, modele, langue)

        for texte in textes_transcrits:
            if texte:
                print(f"\nModèle: {modele}")
                print(texte)

                # Ajouter les résultats dans le tableau récapitulatif
                tableau_recapitulatif.append({
                    'Modèle': modele,
                    'Texte': texte,
                    'Temps (en secondes)': round(time.time() - start_time, 2)  # Temps écoulé
                })

        print(f"Temps total pour le modèle '{modele}': {round(time.time() - start_time, 2)} secondes\n")

    # Créer un tableau récapitulatif et l'afficher
    import pandas as pd
    df_recapitulatif = pd.DataFrame(tableau_recapitulatif)

    return df_recapitulatif


##Traduction

###Prétraitement (Si besoin)

####Si les sous-titres viennent directement de youtube, prétraitement nécessaire pour harmoniser le format

In [ ]:
import re

def convert_srt_to_custom_format(input_file, output_file):
    # Lire le fichier d'entrée
    with open(input_file, 'r', encoding='utf-8') as infile:
        srt_text = infile.read()

    lines = srt_text.strip().split("\n")
    result = []

    # Traitement des lignes du fichier SRT (chaque entrée comprend un numéro, un intervalle de temps et un texte)
    i = 0
    while i < len(lines):
        # Vérification que la ligne contient un numéro et un intervalle de temps
        if i + 1 < len(lines) and " --> " in lines[i + 1]:
            time_range = lines[i + 1]
            start_time, end_time = time_range.split(" --> ")

            # Convertir le format de temps (00:00:00,000 --> 00:00:04,240 en [0.00 - 4.18])
            start_seconds = convert_time_to_seconds(start_time)
            end_seconds = convert_time_to_seconds(end_time)

            # Nettoyer le texte, enlever les balises <font> et les espaces supplémentaires
            if i + 2 < len(lines):
                text = re.sub(r'<font.*?>|</font>', '', lines[i + 2]).strip()
            else:
                text = ""

            # Ajouter le texte converti au résultat
            result.append(f"[{start_seconds:.2f} - {end_seconds:.2f}]  {text}")
            i += 3  # Passer à la prochaine entrée (numéro + time range + texte)
        else:
            i += 1  # Passer à la ligne suivante si ce n'est pas une entrée valide

    # Écrire le résultat dans le fichier de sortie
    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write("\n".join(result))

def convert_time_to_seconds(time_str):
    """ Convertir le temps de HH:MM:SS,MMM en secondes (float). """
    # Remplacer la virgule par un point pour séparer les millisecondes
    time_str = time_str.replace(",", ".")
    time_parts = time_str.split(":")

    # Gestion des cas où l'heure, les minutes ou les secondes sont manquantes
    if len(time_parts) == 3:  # Format HH:MM:SS.MS
        hours, minutes, seconds = map(float, time_parts)
    elif len(time_parts) == 2:  # Format MM:SS.MS
        hours = 0
        minutes, seconds = map(float, time_parts)
    elif len(time_parts) == 1:  # Format SS.MS (rare, mais possible)
        hours = 0
        minutes = 0
        seconds = float(time_parts[0])

    return hours * 3600 + minutes * 60 + seconds


####Chargement modèle traduction

In [ ]:
def charger_et_enregistrer_modele_nllb(chemin_modeles, model_name="facebook/nllb-200-distilled-600M"):
    """
    Charge le modèle NLLB-200 et son tokenizer, et sauvegarde le modèle dans le chemin spécifié.

    Args:
        chemin_modeles (str): Chemin où enregistrer ou charger le modèle.
        model_name (str): Nom du modèle à utiliser.

    Returns:
        tokenizer, model: Tokenizer et modèle chargés.
    """
    chemin_complet = os.path.join(chemin_modeles, model_name.replace('/', '_'))  # Remplace les '/' pour éviter les erreurs de chemin

    if not os.path.exists(chemin_complet):
        os.makedirs(chemin_complet, exist_ok=True)

        # Télécharger le modèle et le tokenizer
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # Enregistrer le modèle et le tokenizer
        tokenizer.save_pretrained(chemin_complet)
        model.save_pretrained(chemin_complet)
        print(f"Modèle téléchargé et enregistré dans {chemin_complet}.")
    else:
        # Charger le modèle et le tokenizer depuis le disque
        tokenizer = AutoTokenizer.from_pretrained(chemin_complet)
        model = AutoModelForSeq2SeqLM.from_pretrained(chemin_complet)
        print(f"Modèle chargé depuis {chemin_complet}.")

    return tokenizer, model


###Traduction concrète avec modèle

#### Avec timecodes

#####Arabe -> Francais par défaut, sinon langues à mettre en paramètre

In [ ]:
def traduire_texte_segment(texte, tokenizer, model, langue_source="arb_Arab", langue_cible="fra_Latn"):
    """
    Traduit un texte donné d'une langue source vers une langue cible.

    Args:
        texte (str): Texte à traduire.
        tokenizer: Tokenizer du modèle de traduction.
        model: Modèle de traduction.
        langue_source (str): Code langue source (ex: "arb").
        langue_cible (str): Code langue cible (ex: "fra").

    Returns:
        str: Texte traduit.
    """
    # Définir les IDs des langues manuellement si `lang_code_to_id` n'est pas disponible
    lang_code_to_id = {
        "arb_Arab": 256011,  # ID pour l'arabe
        "fra_Latn": 256057,   # ID pour le français
        "eng_Latn": 250004   # ID pour l'anglais

    }

    if not texte.strip():  # Ignorer les textes vides
        return ""

    tokenizer.src_lang = langue_source
    encoded_input = tokenizer(texte, return_tensors="pt")
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=lang_code_to_id[langue_cible]
    )
    return tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

def traduire_sous_titres_timecode(lignes, tokenizer, model, langue_source="arb_Arab", langue_cible="fra_Latn"):
    """
    Traduit des sous-titres avec timecodes d'une langue source vers une langue cible.

    Args:
        lignes (list): Liste des lignes du fichier SRT (chaque ligne avec timecode et texte).
        tokenizer: Tokenizer du modèle de traduction.
        model: Modèle de traduction.
        langue_source (str): Code langue source (ex: "arb").
        langue_cible (str): Code langue cible (ex: "fra").

    Returns:
        list: Lignes traduites avec timecodes conservés.
    """
    lignes_traduites = []

    for ligne in lignes:
        match = re.match(r"\[(\d+\.\d+) - (\d+\.\d+)]\s*(.*)", ligne.strip())
        if match:
            start_time = match.group(1)
            end_time = match.group(2)
            texte_original = match.group(3).strip()

            # Traduire le texte
            texte_traduit = traduire_texte_segment(texte_original, tokenizer, model, langue_source, langue_cible)

            # Ajouter la ligne traduite avec le timecode
            lignes_traduites.append(f"[{start_time} - {end_time}]  {texte_traduit}")
        else:
            # Conserver les lignes sans texte (numéros de sous-titres, par exemple)
            lignes_traduites.append(ligne.strip())

    return lignes_traduites


######Test

####Sans timecodes

#####Arabe -> Francais par défaut, sinon langues à mettre en paramètre

In [ ]:
import os

def traduire_fichier_textuel_sans_timecode(tokenizer, model, chemin_entree, langue_source="arb_Arab", langue_cible="fra_Latn", chemin_sortie=None):
    """
    Traduit un fichier texte d'une langue source à une langue cible sans prendre en compte les timecodes.

    Args:
        tokenizer: Le tokenizer NLLB chargé.
        model: Le modèle NLLB chargé.
        chemin_entree (str): Chemin du fichier d'entrée à traduire.
        langue_source (str): Code de langue de la langue source (par défaut "ar").
        langue_cible (str): Code de langue de la langue cible (par défaut "fr").
        chemin_sortie (str, optionnel): Chemin du fichier de sortie. Par défaut, ajoute "_fr" au nom du fichier d'entrée.

    Returns:
        str: Chemin du fichier traduit.
    """
    # Vérification de l'existence du fichier d'entrée
    if not os.path.exists(chemin_entree):
        raise FileNotFoundError(f"Le fichier d'entrée {chemin_entree} n'existe pas.")

    # Lire le contenu du fichier d'entrée
    with open(chemin_entree, "r", encoding="utf-8") as fichier:
        texte = fichier.read()

    # Configurer la langue source pour le tokenizer
    tokenizer.src_lang = langue_source

    # Encoder le texte source pour le modèle
    encoded_input = tokenizer(texte, return_tensors="pt", padding=True, truncation=True)

    # Utiliser lang2id pour récupérer l'ID de langue pour la langue cible
    lang_id = tokenizer.lang2id[langue_cible]

    # Générer la traduction avec le modèle
    output = model.generate(**encoded_input, forced_bos_token_id=lang_id)

    # Décoder le texte traduit
    traduction = tokenizer.batch_decode(output, skip_special_tokens=True)[0]

    # Déterminer le chemin du fichier de sortie
    if chemin_sortie is None:
        base, ext = os.path.splitext(chemin_entree)
        chemin_sortie = f"{base}_traduit_{langue_cible}{ext}"

    # Écrire la traduction dans le fichier de sortie
    with open(chemin_sortie, "w", encoding="utf-8") as sortie_fichier:
        sortie_fichier.write(traduction)

    print(f"Traduction enregistrée dans {chemin_sortie}.")
    return chemin_sortie




###Traduction concrète avec API

In [ ]:
import re
from together import Together

MOTS_ANGLAIS_COMMUNS = set([
    "the", "is", "are", "you", "that", "have", "with", "this",
    "not", "for", "it", "be", "by", "from", "was", "were", "they",
    "their", "them", "his", "her", "would", "could", "should", "will"
])

def contient_arabe(texte):
    return bool(re.search(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\uFDFD\uFDFA]', texte))

def contient_trop_d_anglais(texte, seuil_ratio=0.10):
    mots = re.findall(r'\b\w+\b', texte.lower())
    if not mots:
        return False
    nb_total = len(mots)
    nb_anglais = sum(1 for mot in mots if mot in MOTS_ANGLAIS_COMMUNS)
    return (nb_anglais / nb_total) > seuil_ratio


def Traduire_texte_avec_API(text):
    client = Together(api_key="") #Mettez votre API ici :-)

    lines = text.split("\n")
    group_size = 30
    num_groups = (len(lines) // group_size) + (1 if len(lines) % group_size != 0 else 0)

    all_translated_text = ""

    for i in range(num_groups):
        start_index = i * group_size
        end_index = (i + 1) * group_size
        group_text = "\n".join(lines[start_index:end_index])

        prompt = f"""
        Traduis intégralement ce texte en français **sans modifier sa structure**.

        – **Chaque ligne commence par un timecode** : conserve-le **strictement inchangé**.
        – **Ne modifie pas l’ordre ni le nombre de lignes**. Le texte traduit doit avoir **exactement le même nombre de lignes** que l’original.
        – **Ne fusionne, ne divise, ne reformule, ne supprime aucune ligne**.
        – Traduis uniquement le contenu après chaque timecode. **Aucune explication, commentaire, ou sortie additionnelle n’est autorisée**.
        – **Utilise "Allah"** si "Dieu" fait référence à Allah.
        – **N’utilise pas de caractères arabes, balises HTML ou symboles spéciaux**.
        – Adapte la ponctuation au français **sans changer la structure ou le sens**.

        Voici le texte à traduire :
        {group_text}
        """

        tentative = 0
        max_tentatives = 5
        traduction_valide = False
        texte_traduit = ""

        while not traduction_valide and tentative < max_tentatives:
            response = client.chat.completions.create(
                model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                messages=[{"role": "user", "content": prompt}],
            )
            texte_traduit = re.sub(r"<think>.*?</think>", "", response.choices[0].message.content, flags=re.DOTALL).strip()

            if not contient_arabe(texte_traduit) and not contient_trop_d_anglais(texte_traduit):
                traduction_valide = True
            else:
                tentative += 1
                if tentative < max_tentatives:
                    print(f"Bloc {i+1}/{num_groups} invalide. Nouvelle tentative ({tentative + 1})...")
                else:
                    print(f"Bloc {i+1}/{num_groups} toujours invalide après {max_tentatives} tentatives.")

        all_translated_text += texte_traduit + "\n"
        apercu = "\n".join(texte_traduit.splitlines()[:2])
        print(f"Bloc {i+1}/{num_groups} traité.\nAperçu :\n{apercu}\n")
    return all_translated_text


In [ ]:
def Traduire_titre_avec_API_short(text):
    client = Together(api_key="") #Mettez votre API ici :-)
    prompt = f"Traduis ce titre en français en ne gardant que le titre. \n{text}"

    response = client.chat.completions.create(
        model="deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free",
        messages=[{"role": "user", "content": prompt}],
    )

    translated_text = response.choices[0].message.content

    # Supprimer la section <think>...</think>
    cleaned_text = re.sub(r"<think>.*?</think>", "", translated_text, flags=re.DOTALL).strip()

    return cleaned_text


###Vérifications texte traduit

In [ ]:
def contient_caracteres_arabes_ou_speciaux(texte):
    """
    Détecte si le texte contient des caractères arabes ou des symboles spéciaux
    comme ﷺ, ﷻ, etc.
    """
    return bool(re.search(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\uFDFD\uFDFA]', texte))

###Post-Traitement

####Enlever majuscules inutiles

#####Francais

In [ ]:
import re

def corriger_majuscule(fichier_entree, fichier_sortie=None):
    # Lire le texte depuis le fichier d'entrée
    with open(fichier_entree, 'r', encoding="utf-8") as f:
        texte = f.read()

    # Séparer le texte en segments en utilisant les timestamps
    segments = re.split(r'(\[\d+\.\d+ - \d+\.\d+\]  )', texte)

    # Extraire uniquement les textes des segments (en ignorant les timestamps)
    timestamps = [segments[i] for i in range(1, len(segments), 2)]
    textes = [segments[i] for i in range(2, len(segments), 2)]
    segments = re.split(r'(\[\d+\.\d+ - \d+\.\d+\]  )', texte)

    # Initialisation : le premier texte sera capitalisé
    if len(textes) > 0:
        textes[0] = textes[0].capitalize()

    # Parcourir les autres segments
    for i in range(1, len(textes)):
        # Vérifier si le texte précédent se termine par un signe de ponctuation
        if textes[i - 1].strip()[-1] in '.!?':
            textes[i] = textes[i].capitalize()
        else :
            textes[i] = textes[i].lower()

    # Recréer le texte final avec les timestamps au début de chaque segment
    texte_corrige = ""
    for i in range(len(textes)):
        texte_corrige += timestamps[i] + textes[i]

    # Si le fichier de sortie est None, on utilise le fichier d'entrée
    if fichier_sortie is None:
        fichier_sortie = fichier_entree

    # Écrire le texte corrigé dans le fichier de sortie
    with open(fichier_sortie, 'w', encoding='utf-8') as f:
        f.write(texte_corrige)



##Vérifications avant montage


###Sous fonctions

####Afficher audio

In [ ]:
def afficher_audio(nom_fichier_mp3):
  display(Audio(nom_fichier_mp3, autoplay=False))



# afficher_audio(temp_audio_path)

####Fenêtre pour modifier texte

In [ ]:
# Fonction pour ouvrir et modifier un fichier existant
def modifier_fichier(nom_fichier):
    # Lire le contenu du fichier existant
    try:
        with open(nom_fichier, 'r') as f:
            contenu = f.read()
    except FileNotFoundError:
        print(f"Le fichier {nom_fichier} n'existe pas.")
        return

    # Créer une zone de texte pour modifier le contenu
    text_area = widgets.Textarea(
        value=contenu,
        placeholder='Modifier le contenu ici...',
        description=f'{nom_fichier}:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='300px')
    )

    # Créer un bouton pour sauvegarder les modifications
    save_button = widgets.Button(description="Sauvegarder les modifications")

    # Fonction de sauvegarde
    def save_changes(b):
        modified_content = text_area.value
        with open(nom_fichier, 'w') as f:
            f.write(modified_content)
        print(f"Le fichier {nom_fichier} a été sauvegardé avec les modifications.")

    save_button.on_click(save_changes)

    # Afficher la zone de texte et le bouton
    display(text_area, save_button)

# # Exemple d'utilisation avec un fichier existant
# modifier_fichier(subtitles_path_fr)


###Modifier textes avec vocal et titre de la vidéo

In [ ]:
import os
import shutil  # Pour la suppression de dossiers
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Audio

def interface_modifier_fichier_et_jouer_mp3(nom_fichier_texte, nom_fichier_mp3, titre_modifiable="", chemin_texte_transcris=""):
    """
    Interface pour modifier un fichier texte, afficher un texte non modifiable, et jouer un fichier MP3.
    :param nom_fichier_texte: Chemin du fichier texte à modifier.
    :param nom_fichier_mp3: Chemin du fichier MP3 à jouer.
    :param titre_modifiable: Le titre modifiable qui sera affiché et mis à jour.
    :param chemin_texte_transcris: Chemin du fichier dont le contenu sera affiché en lecture seule.
    """

    # Lire le contenu du fichier texte existant
    try:
        with open(nom_fichier_texte, 'r') as f:
            contenu = f.read()
    except FileNotFoundError:
        print(f"Le fichier {nom_fichier_texte} n'existe pas.")
        return

    # Lire le contenu du fichier texte_transcris (non modifiable)
    try:
        with open(chemin_texte_transcris, 'r') as f:
            contenu_transcris = f.read()
    except FileNotFoundError:
        contenu_transcris = "Le fichier texte transcrit est introuvable."

    # Créer une zone de texte pour modifier le contenu
    text_area = widgets.Textarea(
        value=contenu,
        placeholder='Modifier le contenu ici...',
        description='Sous-titres:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='300px')
    )

    # Créer un champ pour modifier le titre (avec le titre passé en paramètre)
    title_input = widgets.Text(
        value=titre_modifiable,
        placeholder="Titre modifiable ici...",
        description="Titre:",
        disabled=False,
        layout=widgets.Layout(width='100%')
    )

    # Créer une zone de texte non modifiable pour afficher texte_transcris
    text_transcris_widget = widgets.Textarea(
        value=contenu_transcris,
        placeholder="Contenu affiché ici...",
        description="Texte transcrit:",
        disabled=True,
        layout=widgets.Layout(width='100%', height='200px')
    )

    # Variable pour enregistrer le titre modifié
    modified_title = ""

    # Créer un bouton pour sauvegarder les modifications (style rouge, taille réduite)
    save_button = widgets.Button(
        description="Sauvegarder",
        button_style="success",  # Style vert pour le bouton
        layout=widgets.Layout(width="30%")  # Réduction de la taille du bouton
    )

    # Créer un bouton pour ignorer ce dossier
    ignore_button = widgets.Button(
        description="Ignorer ce dossier",
        button_style="warning",  # Style orange pour le bouton
        layout=widgets.Layout(width="30%")
    )

    # Créer un bouton pour effacer ce dossier
    delete_button = widgets.Button(
        description="Effacer ce dossier",
        button_style="danger",  # Style rouge pour le bouton
        layout=widgets.Layout(width="30%")
    )

    # Fonction de sauvegarde
    def save_changes(b):
        nonlocal modified_title
        modified_content = text_area.value
        modified_title = title_input.value  # Enregistrer le titre modifié
        with open(nom_fichier_texte, 'w') as f:
            f.write(modified_content)
        print(f"Le fichier {nom_fichier_texte} a été sauvegardé avec les modifications.")

        # Créer le fichier avec le titre modifié comme nom de fichier, mais vide
        directory = os.path.dirname(nom_fichier_texte)  # Récupérer le répertoire du fichier

        # Créer le fichier vierge avec l'extension _titre.txt
        titre_fichier = f"{modified_title}_titre.txt"
        titre_nom_fichier = os.path.join(directory, titre_fichier)

        # Créer un fichier vierge
        with open(titre_nom_fichier, 'w') as f:
            pass  # Laisser le fichier vide

        # Créer un fichier de révision avec le contenu modifié, et l'extension _revise.txt
        revise_fichier = f"{modified_title}_revise.txt"
        revise_nom_fichier = os.path.join(directory, revise_fichier)

        # Sauvegarder le contenu modifié dans le fichier _revise.txt
        with open(revise_nom_fichier, 'w') as f:
            f.write(modified_content)

        # Afficher les fichiers créés
        print(f"Fichier créé : {titre_nom_fichier}")
        print(f"Fichier révisé sauvegardé : {revise_nom_fichier}")

        # Cacher les widgets liés à cette interface après la sauvegarde
        interface_container.layout.display = 'none'
        audio_container.layout.display = 'none'

    # Fonction pour ignorer ce dossier
    def ignore_folder(b):
        print("Dossier ignoré.")
        interface_container.layout.display = 'none'
        audio_container.layout.display = 'none'

    # Fonction pour effacer ce dossier
    def delete_folder(b):
        directory = os.path.dirname(nom_fichier_texte)
        if os.path.exists(directory):
            shutil.rmtree(directory)  # Supprimer le dossier et son contenu
            print(f"Dossier {directory} effacé.")
        else:
            print(f"Le dossier {directory} n'existe pas.")
        interface_container.layout.display = 'none'
        audio_container.layout.display = 'none'

    # Associer les fonctions aux boutons
    save_button.on_click(save_changes)
    ignore_button.on_click(ignore_folder)
    delete_button.on_click(delete_folder)

    # Lire le fichier MP3 et le passer à l'argument `value`
    with open(nom_fichier_mp3, 'rb') as f:
        audio_data = f.read()

    # Créer le widget audio avec les données du fichier
    audio_widget = Audio(value=audio_data, autoplay=False)

    # Créer un conteneur pour le lecteur audio
    audio_container = widgets.VBox([audio_widget])

    # Créer un conteneur horizontal pour les boutons
    button_container = widgets.HBox([save_button, ignore_button, delete_button])

    # Créer un conteneur pour l'interface principale
    interface_container = widgets.VBox([
        audio_container,          # Placer le lecteur audio en premier
        title_input,              # Champ modifiable pour le titre
        text_area,                # Zone modifiable pour le contenu
        button_container,         # Conteneur horizontal pour les boutons
        text_transcris_widget     # Texte non modifiable (affiché en dessous)
    ])

    # Afficher l'interface complète
    display(interface_container)

##Montage

###Ajout de sous-titres

####Ajout de sous-titres avec timecode

In [ ]:
from PIL import ImageFont, ImageDraw, Image
import cv2
import os
import numpy as np
from bidi.algorithm import get_display


# Fonction pour ajouter du texte à la vidéo avec OpenCV et Pillow
# Fonction pour ajouter du texte à la vidéo avec OpenCV et Pillow
def ajouter_texte_video_opencv(video_path, texte_path, font_path = police_path, output_path=None):
    if output_path is None:
      file_name, file_extension = os.path.splitext(video_path)
      output_path = f"{file_name}_texte{file_extension}"

    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Erreur lors de l'ouverture de la vidéo.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    try:
        font_size = int(frame_height / 35)
        font = ImageFont.truetype(font_path, size=font_size)
    except IOError:
        print("Erreur : Impossible de charger la police. Vérifiez le chemin.")
        return

    with open(texte_path, 'r', encoding='utf-8') as f:
        lignes = f.readlines()

    annotations = []
    for ligne in lignes:
        if ligne.strip() == "":
            continue
        parts = ligne.split(']')
        timecode_str, texte = parts[0][1:], parts[1].strip()
        start_time, end_time = map(float, timecode_str.split(' - '))
        annotations.append((start_time, end_time, texte))

    def couper_texte(texte, font, max_width):
        words = texte.split()
        lines = []
        current_line = ""

        for word in words:
            test_line = current_line + (" " if current_line else "") + word
            bbox = font.getbbox(test_line)
            width, _ = bbox[2], bbox[3]
            if width <= max_width:
                current_line = test_line
            else:
                if current_line:
                    lines.append(current_line)
                current_line = word
        if current_line:
            lines.append(current_line)
        return lines

    # Fonction pour appliquer BiDi correctement
    def apply_bidi_correctly(texte):
        segments = []
        current_segment = ""
        current_script = None

        for char in texte:
            if "\u0600" <= char <= "\u06FF" or "\u0750" <= char <= "\u077F" or "\u08A0" <= char <= "\u08FF":
                script = "arabic"
            else:
                script = "latin"

            if script != current_script:
                if current_segment:
                    segments.append((current_script, current_segment))
                current_segment = char
                current_script = script
            else:
                current_segment += char

        if current_segment:
            segments.append((current_script, current_segment))

        processed_segments = []
        for script, segment in segments:
            if script == "arabic":
                processed_segments.append(get_display(segment[::-1]))  # Corriger l'ordre arabe
            else:
                processed_segments.append(segment)

        return "".join(processed_segments)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_time = frame_idx / fps

        for start_time, end_time, texte in annotations:
            if start_time <= current_time <= end_time:
                pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
                draw = ImageDraw.Draw(pil_image)

                texte_bidi = apply_bidi_correctly(texte)

                max_width = frame_width - 180
                lines = couper_texte(texte_bidi, font, max_width)

                total_height = sum([font.getbbox(line)[3] - font.getbbox(line)[1] for line in lines if line.strip()]) + (len(lines) - 1) * 25
                y_position = frame_height - int(frame_height * 0.25) - total_height

                words = texte.split()
                num_words = len(words)

                # Calculer la durée par mot
                duration_per_word = (end_time - start_time) / num_words
                current_word_start_time = start_time

                for line in lines:
                    if line.strip() == "":
                        continue

                    text_width, text_height = font.getbbox(line)[2], font.getbbox(line)[3] - font.getbbox(line)[1]
                    background_position = ((frame_width - text_width) // 2 - 10, y_position - 10)
                    background_size = (text_width + 20, text_height + 40)
                    draw.rectangle([background_position, (background_position[0] + background_size[0], background_position[1] + background_size[1])], fill=(0, 0, 0, 128))

                    # Découper la ligne en mots et appliquer l'effet rouge uniquement pendant le temps du mot
                    x_position = (frame_width - text_width) // 2
                    words_in_line = line.split()

                    for word in words_in_line:
                        word_width, word_height = font.getbbox(word)[2], font.getbbox(word)[3] - font.getbbox(word)[1]

                        # Vérifier si le mot doit être affiché en rouge
                        if current_word_start_time <= current_time <= current_word_start_time + duration_per_word:
                            word_color = (255, 215, 0)  # Doré
                        else:
                            word_color = (255, 255, 255)  # Blanc

                        # Dessiner chaque mot avec la couleur appropriée
                        draw.text((x_position, y_position), word, font=font, fill=word_color)
                        x_position += word_width + 10  # Décalage pour le mot suivant

                        # Passer à l'intervalle suivant pour le mot
                        current_word_start_time += duration_per_word

                    y_position += text_height + 25

                frame = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
                break

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Vidéo exportée sous : {output_path}")
    return output_path


###Ajout de petit logo après l'affichage du titre

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
import subprocess

def ajouter_logo_video(video_path, logo_path=logo_path, follow_path=follow_path, output_path=None, title_duration=0, message_abonnement_duration=message_abonnement_duration):
    """
    Ajoute un logo centré horizontalement à la vidéo, puis le remplace par une autre image.

    :param video_path: Chemin vers la vidéo à traiter
    :param logo_path: Chemin vers l'image (logo) initiale à ajouter
    :param follow_path: Chemin vers l'image qui remplace le logo
    :param output_path: Chemin vers la vidéo de sortie
    :param title_duration: Durée en secondes avant l'affichage du logo
    :param message_abonnement_duration: Durée avant la fin pour afficher follow_path
    :return: Le chemin vers la vidéo de sortie
    """
    if not os.path.exists(video_path):
        print(f"Erreur : fichier vidéo introuvable ({video_path}).")
        return None

    if not os.path.exists(logo_path):
        print(f"Erreur : fichier logo introuvable ({logo_path}).")
        return None

    if not os.path.exists(follow_path):
        print(f"Erreur : fichier follow introuvable ({follow_path}).")
        return None

    if output_path is None:
        file_name, file_extension = os.path.splitext(video_path)
        output_path = f"{file_name}_logo{file_extension}"
        print("Output_logo_ajouté = ", output_path)

    # Charger la vidéo
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Erreur lors de l'ouverture de la vidéo.")
        return None

    # Obtenir les propriétés de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Créer un objet VideoWriter pour enregistrer la vidéo de sortie
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Charger les images du logo et de l'image de suivi
    logo = Image.open(logo_path).convert("RGBA")
    follow_image = Image.open(follow_path).convert("RGBA")

    # Redimensionner les images (environ 30% de la largeur de la vidéo)
    logo_size = int(frame_width * 0.30)  # 30% de la largeur de la vidéo
    logo_resized = logo.resize((logo_size, int(logo.size[1] * (logo_size / logo.size[0]))), Image.Resampling.LANCZOS)
    follow_resized = follow_image.resize(
        (logo_size, int(follow_image.size[1] * (logo_size / follow_image.size[0]))),
        Image.Resampling.LANCZOS
    )

    # # Appliquer une transparence au logo
    # alpha = 0.5  # Opacité à 50 %
    # transparent_logo = logo_resized.copy()
    # alpha_layer = transparent_logo.getchannel('A')
    # alpha_layer = ImageEnhance.Brightness(alpha_layer).enhance(alpha)
    # transparent_logo.putalpha(alpha_layer)


    # Calculer la position centrée horizontalement et à 30 pixels du haut
# Calculer la position centrée horizontalement et à 30 pixels du bas
    position = ((frame_width - logo_resized.width) // 2, frame_height - logo_resized.height - 50)

    # Calculer les frames pour les durées spécifiques
    logo_start_frame = int(fps * title_duration)
    follow_start_frame = frame_count - int(fps * message_abonnement_duration)

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir la frame OpenCV (BGR) en image PIL (RGB)
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Ajouter uniquement une des deux images
        if frame_idx >= follow_start_frame:
            pil_image.paste(follow_resized, position, follow_resized)
        elif frame_idx >= logo_start_frame:
            pil_image.paste(logo_resized, position, logo_resized)

        # Reconvertir en frame OpenCV
        frame = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)

        # Écrire la frame dans la vidéo de sortie
        out.write(frame)
        frame_idx += 1

    # Libérer les objets
    cap.release()
    out.release()
    print(f"Vidéo avec logo et image de suivi exportée sous : {output_path}")
    return output_path


# # Exemple d'utilisation
# # Ajouter le logo à la vidéo
# video_avec_logo = ajouter_logo_video(chemin_video, logo_path)

# # Fusionner la vidéo avec l'audio
# if verifier_fichier_audio(audio_path):
#     fusionner_audio_video(video_avec_logo, audio_path, video_avec_logo.replace(".mp4", "_final.mp4"))


In [ ]:
from IPython.display import HTML
from base64 import b64encode

def afficher_video_colab(video_path):
    """
    Affiche une vidéo dans une cellule Colab en utilisant un widget HTML.

    :param video_path: Chemin du fichier vidéo à afficher.
    """
    # Encodage de la vidéo en base64 pour l'afficher dans HTML
    with open(video_path, "rb") as video_file:
        video_data = video_file.read()
        video_base64 = b64encode(video_data).decode()

    # Générer le code HTML pour afficher la vidéo
    video_html = f"""
    <video width="640" height="360" controls>
        <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
        Votre navigateur ne supporte pas l'affichage des vidéos.
    </video>
    """
    return HTML(video_html)


# # Afficher la vidéo
# # Exemple d'utilisation
# # Remplacez ce chemin par le chemin réel de votre vidéo

# afficher_video_colab(video_path)


###Remettre l'audio dans la vidéo

In [ ]:
# Fonction pour fusionner la vidéo avec l'audio et convertir la vidéo en H264
def fusionner_audio_video(video_path, audio_path, output_path):
    # Commande ffmpeg pour fusionner la vidéo et l'audio avec conversion en H264
    command = [
        'ffmpeg', '-y', '-i', video_path, '-i', audio_path,
        '-c:v', 'libx264', '-c:a', 'aac', '-strict', 'experimental', output_path
    ]

    # Exécution de la commande ffmpeg avec gestion des erreurs
    try:
        print(f"Running command: {' '.join(command)}")
        subprocess.run(command, check=True, stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        print(f"Vidéo avec audio exportée sous : {output_path}")
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de la fusion de l'audio et de la vidéo : {e}")
        print(f"stderr: {e.stderr.decode()}")
        print(f"stdout: {e.stdout.decode()}")

# Vérifier si le fichier audio existe
def verifier_fichier_audio(audio_path):
    if os.path.exists(audio_path):
        print("Le fichier audio existe.")
    else:
        print("Le fichier audio n'existe pas.")
        return False
    return True


In [ ]:
import cv2
import numpy as np
import os

def ajouter_rectangle_noir_video(video_path, police, output_path=None):
    """
    Ajoute un rectangle noir semi-transparent sur la moitié inférieure de la vidéo,
    avec un dégradé progressif sur les 5% supérieurs.

    :param video_path: Chemin vers la vidéo à traiter.
    :param police: Chemin vers la police à utiliser (même si le texte ne change pas de position).
    :param output_path: Chemin vers la vidéo de sortie.
    :return: Le chemin vers la vidéo de sortie.
    """
    if output_path is None:
        file_name, file_extension = os.path.splitext(video_path)
        output_path = f"{file_name}_rectangle{file_extension}"

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Erreur lors de l'ouverture de la vidéo.")
        return

    # Obtenir les propriétés de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Définir la hauteur du rectangle (tiers inférieur de la vidéo)
    rect_height = frame_height // 3
    gradient_height = int(rect_height * 0.05)  # 5% de la hauteur du rectangle pour le dégradé

    # Création du rectangle noir avec dégradé
    rect = np.zeros((rect_height, frame_width, 3), dtype=np.uint8)

    # Ajouter le dégradé sur les 5% supérieurs du rectangle
    for y in range(gradient_height):
        alpha = 1 - (y / gradient_height)  # Dégradé vers le noir
        rect[y, :, :] = (alpha * 255, alpha * 255, alpha * 255)  # Dégradé vers le noir

    # Remplir la partie inférieure du rectangle avec du noir pur (après le dégradé)
    rect[gradient_height:, :, :] = (0, 0, 0)  # Partie noire pure

    # Créer l'objet VideoWriter pour écrire la vidéo de sortie
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Ajouter le rectangle noir avec dégradé sur la moitié inférieure de chaque frame
        frame[frame_height - rect_height:, :] = rect  # Ajouter directement le rectangle sur la vidéo

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    print(f"Vidéo avec rectangle noir et dégradé exportée sous : {output_path}")
    return output_path


###Ajout du titre avant l'apparition du logo

In [ ]:
import os
import cv2
from PIL import Image, ImageDraw, ImageFont
import numpy as np

def ajouter_titre_debut_video(video_path, titre, police = police_path, output_path=None, title_duration=title_duration):
    """
    Ajoute un titre de 3 secondes au début de la vidéo, centré sur un bandeau rouge autour du texte,
    avec le texte en blanc, directement sur la vidéo originale. Le texte va automatiquement à la ligne si nécessaire.

    :param video_path: Chemin vers la vidéo à traiter.
    :param titre: Le titre à afficher pendant 3 secondes.
    :param police: Chemin vers la police à utiliser pour le texte.
    :param output_path: Chemin vers la vidéo de sortie.
    :param title_duration: Durée du titre en secondes.
    :return: Le chemin vers la vidéo de sortie.
    """
    # Si aucun chemin de sortie n'est spécifié, générer un nom de fichier avec _titre_ajoute
    if output_path is None:
        file_name, file_extension = os.path.splitext(video_path)
        output_path = f"{file_name}_titre{file_extension}"

    # Charger la vidéo
    cap = cv2.VideoCapture(video_path)

    # Vérifier si la vidéo a été correctement ouverte
    if not cap.isOpened():
        print("Erreur lors de l'ouverture de la vidéo.")
        return

    # Obtenir les propriétés de la vidéo
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Créer un objet VideoWriter pour enregistrer la vidéo de sortie
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    # Paramètres du texte et du bandeau
    title_frame_count = int(fps * title_duration)  # Nombre de frames pour le titre

    # Charger la police avec une taille fixe
    font_size = 40
    try:
        font = ImageFont.truetype(police, font_size)
    except IOError:
        print("La police spécifiée n'a pas été trouvée, utilisation de la police par défaut.")
        font = ImageFont.load_default()

    # Diviser le texte en lignes pour qu'il tienne dans la largeur de la vidéo
    def diviser_en_lignes(titre, largeur_max):
        words = titre.split(' ')
        lignes = []
        ligne_actuelle = ""

        for mot in words:
            test_ligne = ligne_actuelle + " " + mot if ligne_actuelle else mot
            text_width = font.getbbox(test_ligne)[2]  # Utilisation de getbbox pour obtenir la largeur
            if text_width <= largeur_max:
                ligne_actuelle = test_ligne
            else:
                lignes.append(ligne_actuelle)
                ligne_actuelle = mot

        if ligne_actuelle:
            lignes.append(ligne_actuelle)

        return lignes

    max_text_width = int(frame_width * 0.8)  # Texte limité à 80% de la largeur de la vidéo
    lignes = diviser_en_lignes(titre, max_text_width)

    # Calculer la hauteur totale des lignes de texte
    text_height_total = sum(font.getbbox(ligne)[3] - font.getbbox(ligne)[1] for ligne in lignes) + 20 * (len(lignes) - 1)

    # Paramètres du bandeau
    bandeau_padding = 20
    bandeau_padding_top = 10  # Espacement au-dessus du texte dans le bandeau
    bandeau_padding_bottom = 20  # Espacement en bas

    # Calculer la taille et la position du bandeau
    bandeau_width = max_text_width + 2 * bandeau_padding
    bandeau_height = text_height_total + bandeau_padding_top + bandeau_padding_bottom
    bandeau_x = (frame_width - bandeau_width) // 2
    bandeau_y = int(frame_height * 0.3)

    # Créer le bandeau (fond rouge)
    bandeau = Image.new("RGB", (bandeau_width, bandeau_height), color=(255, 0, 0))  # Fond rouge
    draw_bandeau = ImageDraw.Draw(bandeau)

    # Dessiner chaque ligne de texte sur le bandeau
    y_offset = bandeau_padding_top
    for ligne in lignes:
        text_bbox = font.getbbox(ligne)
        text_width = text_bbox[2] - text_bbox[0]
        x_position = (bandeau_width - text_width) // 2
        draw_bandeau.text((x_position, y_offset), ligne, font=font, fill=(255, 255, 255))  # Texte blanc
        y_offset += text_bbox[3] - text_bbox[1] + 20  # Ajouter un espacement entre les lignes

    # Convertir le bandeau en image OpenCV
    bandeau_frame = cv2.cvtColor(np.array(bandeau), cv2.COLOR_RGB2BGR)

    # Ajouter le bandeau sur les frames originales pendant la durée du titre
    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx < title_frame_count:
            # Placer le bandeau sur la frame originale
            frame[bandeau_y:bandeau_y + bandeau_height, bandeau_x:bandeau_x + bandeau_width] = bandeau_frame

        out.write(frame)
        frame_idx += 1

    # Libérer les objets
    cap.release()
    out.release()
    print(f"Vidéo avec titre ajoutée exportée sous : {output_path}")
    return output_path





#Execution concrète de code

##Sous-Sections

###Pretraitement

####Générations de sous-dossier

In [ ]:
liste_videos = lister_elements_videos_dossier(chemin_dossier_0)
for video in liste_videos:
    print(video)
    generer_dossier_et_deplacer_video_dedans(video)
liste_dossiers_videos=lister_sous_dossiers_avec_videos(chemin_dossier_0)
for dossier in liste_dossiers_videos :
    deplacer_dossier(dossier, chemin_dossier_1)


####Extraction de sous-titres

#####Avec time-codes

In [ ]:
# Liste des sous-dossiers contenant des vidéos
liste_sous_dossiers = lister_sous_dossiers_avec_videos(chemin_dossier_1)
print(len(liste_sous_dossiers))
modele_whisper = charger_modele_textuel(chemin_modeles, modele=modele_textuel_choisi)

# Liste pour stocker les chemins des fichiers audio extraits
liste_audios = []


chemins_videos = lister_videos_dans_sous_dossiers_uniquement(chemin_dossier_1)  # Renvoie une liste des chemins de vidéos
print(len(chemins_videos))

# Extraire l'audio de chaque vidéo
for chemin_video in chemins_videos:
    # Déterminer le chemin de l'audio à extraire (par exemple, changer l'extension .mp4 en .mp3)
    chemin_audio = chemin_video.replace(".mp4", ".mp3")  # Remplacez ".mp4" par l'extension de votre vidéo si nécessaire

    # Vérifier si le fichier audio existe déjà avant d'extraire
    if os.path.exists(chemin_audio):
        # print(f"L'audio {chemin_audio} existe déjà. Il ne sera pas extrait.")
        liste_audios.append(chemin_audio)  # Ajouter l'audio existant à la liste sans l'extraire
    else:
        # Extraire l'audio de la vidéo si le fichier audio n'existe pas
        time1 = time.time()
        audio = extraire_audio_de_video(chemin_video)
        time2 = time.time()
        time_fin = time2 - time1
        if audio:  # Vérifier si l'extraction a réussi
            liste_audios.append(audio)
            print(f"L'audio {chemin_audio} A été créé.")
            print("Temps transcription = ", time_fin)
        else:
            print(f"Échec de l'extraction audio pour : {chemin_video}")

# Appliquer la transcription sur chaque fichier audio extrait
if liste_audios:
    transcriptions = []  # Liste pour stocker les transcriptions
    debut = time.time()  # Chronométrer le temps global

    # Transcrire chaque fichier audio
    for audio in liste_audios:
        transcription = transcrire_audio_vers_texte_avec_code_temps(audio, modele_whisper)
        transcriptions.append(transcription)

    fin = time.time()  # Temps total pour la transcription
    print(f"Temps pour la transcription de {len(liste_audios)} fichiers audio : {fin - debut:.2f} secondes")

    # Afficher les résultats
    for audio, transcription in zip(liste_audios, transcriptions):
        if isinstance(transcription, str):
            print(f"Transcription réussie pour {audio}")
        else:
            print(f"Transcription échouée pour {audio}")
else:
    print("Aucun fichier audio n'a été extrait pour transcription.")



In [ ]:
liste_dossiers_videos=lister_sous_dossiers_avec_videos(chemin_dossier_1)
for dossier in liste_dossiers_videos :
    if recuperer_fichiers_par_suffixe(dossier, extension_transcription_avec_timecode) != []:
        print(dossier)
        deplacer_dossier(dossier, chemin_dossier_2)

#####Sans timecodes

In [ ]:
# liste_sous_dossiers =lister_sous_dossiers_avec_videos(chemin_dossier_1)
# modele_whisper = charger_modele_textuel(chemin_modeles)
# # Initialiser une liste pour stocker les chemins des fichiers audio extraits
# liste_audios = []

# # Parcourir les sous-dossiers contenant des vidéos
# for dossier in liste_sous_dossiers:
#     # Lister toutes les vidéos dans le sous-dossier
#     chemins_videos = lister_elements_videos_dossier(dossier)  # Renvoie une liste des chemins de vidéos
#     for chemin_video in chemins_videos:
#         print(f"Traitement de la vidéo : {chemin_video}")

#         # Extraire l'audio de la vidéo
#         audio = extraire_audio_de_video(chemin_video)
#         if audio:  # Vérifier si l'extraction a réussi
#             print(f"Audio extrait : {audio}")
#             liste_audios.append(audio)
#         else:
#             print(f"Échec de l'extraction audio pour : {chemin_video}")

# # Appliquer la transcription sur chaque fichier audio extrait
# if liste_audios:
#     print(f"Nombre total d'audios à transcrire : {len(liste_audios)}")
#     transcriptions = transcrire_audio_vers_texte_sans_code_temps(liste_audios, modele_whisper)
#     for audio, transcription in zip(liste_audios, transcriptions):
#         if transcription:
#             print(f"Transcription réussie pour {audio}")
#         else:
#             print(f"Transcription échouée pour {audio}")
# else:
#     print("Aucun fichier audio n'a été extrait pour transcription.")



# # liste_dossiers_videos=lister_sous_dossiers_avec_videos(chemin_dossier_1)
# for dossier in liste_dossiers_videos :
#     deplacer_dossier(dossier, chemin_dossier_2)


####Traduction

#####Avec timecodes

In [ ]:
# import time
# import os

# # Charger la liste des sous-dossiers contenant des vidéos
# liste_sous_dossiers = lister_sous_dossiers_avec_videos(chemin_dossier_2)

# # Charger le modèle et le tokenizer une seule fois
# tokenizer, model = charger_et_enregistrer_modele_nllb(chemin_modeles)

# # Traiter chaque sous-dossier
# for sous_dossier in liste_sous_dossiers:
#     try:
#         # Localiser les fichiers nécessaires
#         chemin_video = lister_elements_videos_dossier(sous_dossier)[0]
#         audio = trouver_audio_dans_dossier(sous_dossier)
#         sous_titre = recuperer_fichiers_par_suffixe(sous_dossier, extension_transcription_avec_timecode)[0]

#         # Afficher les détails pour le suivi
#         # Ouvrir le fichier de sous-titres
#         with open(sous_titre, "r", encoding="utf-8") as f:
#             lignes = f.readlines()

#         # Traduire les sous-titres
#         lignes_traduites = []

#         # Mesurer le temps de traduction
#         debut = time.time()
#         # lignes_traduites = traduire_sous_titres_timecode(lignes, tokenizer, model)
#         lignes_traduites = ["A remplir"]
#         fin = time.time()

#         # Enregistrer les lignes traduites dans un nouveau fichier
#         nom_fichier_sortie = sous_titre.replace(extension_transcription_avec_timecode, extension_traduit)
#         with open(nom_fichier_sortie, "w", encoding="utf-8") as f:
#             f.write("\n".join(lignes_traduites))
#         corriger_majuscule(nom_fichier_sortie)
#         print(f"Temps de traduction : {fin - debut:.2f} secondes\n")

#     except IndexError as e:
#         print(f"Erreur : Aucun fichier valide trouvé dans le sous-dossier '{sous_dossier}'. Détails : {e}")
#     except Exception as e:
#         print(f"Erreur inattendue lors du traitement du sous-dossier '{sous_dossier}'. Détails : {e}")



#####Sans timecodes

In [ ]:
# #Penser à n'appeler le modèle qu'ici et l'enlever de la fonction, quitte à voir s'il faut le mettre en paramètre
# liste_sous_dossiers =lister_sous_dossiers_avec_videos(chemin_dossier_2)
# print(liste_sous_dossiers)

# tokenizer, model =charger_et_enregistrer_modele_nllb(chemin_modeles)


# for sous_dossier in liste_sous_dossiers :
#     chemin_video = lister_elements_videos_dossier(sous_dossier)[0]
#     titre_arabe = extract_title_from_path(chemin_video)
#     audio = trouver_audio_dans_dossier(sous_dossier)
#     sous_titre = recuperer_fichiers_par_suffixe(sous_dossier, extension_transcription_sans_timecode)[0]
#     print(sous_titre)

# liste_dossiers_videos=lister_sous_dossiers_avec_videos(chemin_dossier_2)
# for dossier in liste_dossiers_videos :
#     deplacer_dossier(dossier, chemin_dossier_3)


#####Traduction avec API

In [ ]:
import time
import os

liste_sous_dossiers = lister_sous_dossiers_avec_videos(chemin_dossier_2)
total_sous_dossiers = len(liste_sous_dossiers)

for index, sous_dossier in enumerate(liste_sous_dossiers, start=1):
    start_time = time.time()  # Démarrer le chronomètre

    try:
        # Obtenir uniquement le nom du sous-dossier (dernière partie du chemin)
        nom_sous_dossier = os.path.basename(sous_dossier)

        sous_titre = recuperer_fichiers_par_suffixe(sous_dossier, extension_transcription_avec_timecode)[0]

        # Lire le contenu du fichier
        with open(sous_titre, "r", encoding="utf-8") as f:
            texte_a_traduire = f.read()
        # print("Texte à traduire :")
        # print(texte_a_traduire)
        # Traduire le texte
        texte_traduit = Traduire_texte_avec_API(texte_a_traduire)
        # print("Texte_traduit :")
        # print(texte_traduit)
        # Enregistrer la traduction
        nom_fichier_sortie = sous_titre.replace(extension_transcription_avec_timecode, extension_traduit)
        with open(nom_fichier_sortie, "w", encoding="utf-8") as f:
            f.write(texte_traduit)
        # corriger_majuscule(nom_fichier_sortie)

        # Calculer le temps d'exécution
        execution_time = time.time() - start_time  # Temps d'exécution en secondes

        # Afficher un message avec le nombre de sous-dossiers traités et restants
        sous_dossiers_restants = total_sous_dossiers - index
        print(f"Traitement du dossier '{nom_sous_dossier}' terminé. Temps d'exécution : {execution_time:.2f} secondes. "
              f"Il reste {sous_dossiers_restants} sous-dossiers à traiter.")

    except Exception as e:
        print(f"Erreur lors de la traduction du fichier {sous_titre} : {e}")


#####Déplacements

######Déplacement vers dossiers suivants

In [ ]:
liste_dossiers_videos=lister_sous_dossiers_avec_videos(chemin_dossier_2)
for dossier in liste_dossiers_videos :
    if recuperer_fichiers_par_suffixe(dossier, extension_traduit) != []:
        print(dossier)
        deplacer_dossier(dossier, chemin_dossier_3)

######Vérification de la traduction

In [ ]:
import re

# Liste des dossiers contenant des vidéos
liste_dossiers_videos = lister_sous_dossiers_avec_videos(chemin_dossier_3)

# Compteurs pour les statistiques
dossiers_deplaces = 0
dossiers_total = len(liste_dossiers_videos)

for dossier in liste_dossiers_videos:
    # Récupérer les fichiers traduits dans le dossier
    fichiers_traduit = recuperer_fichiers_par_suffixe(dossier, extension_traduit)

    if fichiers_traduit:
        try:
            # Lire le contenu du premier fichier traduit
            with open(fichiers_traduit[0], "r", encoding="utf-8") as f:
                contenu = f.read()

            # Vérifier si le contenu est vide ou contient des caractères arabes ou spéciaux
            if contenu.strip() == "" or contient_caracteres_arabes_ou_speciaux(contenu):
                print(f"Dossier '{dossier}' sera déplacé.")
                deplacer_dossier(dossier, chemin_dossier_2)
                dossiers_deplaces += 1

        except Exception as e:
            print(f"Erreur lors de la lecture du fichier {fichiers_traduit[0]} : {e}")

# Afficher les statistiques
if dossiers_total > 0:
    pourcentage_deplaces = (dossiers_deplaces / dossiers_total) * 100
    print(f"Pourcentage de dossiers déplacés : {pourcentage_deplaces:.2f}%")
else:
    print("Aucun dossier à traiter.")


###Vérifications

In [ ]:
liste_sous_dossiers = lister_sous_dossiers_avec_videos(chemin_dossier_3)
# Mélanger la liste de manière aléatoire
random.shuffle(liste_sous_dossiers)


# Limiter l'itération aux 50 premiers éléments
for sous_dossier in liste_sous_dossiers[:30]:
    try:
        # Localiser les fichiers nécessaires
        chemin_video = lister_elements_videos_dossier(sous_dossier)[0]
        titre_arabe, extension = extract_title_from_path(chemin_video)
        print(titre_arabe)
        texte_transcris = recuperer_fichiers_par_suffixe(sous_dossier, extension_transcription_avec_timecode)[0]
        if (texte_transcris == []):
            texte_transcris = recuperer_fichiers_par_suffixe(sous_dossier, extension_transcription_sans_timecode)[0]
        audio = trouver_audio_dans_dossier(sous_dossier)
        sous_titre = recuperer_fichiers_par_suffixe(sous_dossier, extension_traduit)[0]
        print(sous_titre)

        interface_modifier_fichier_et_jouer_mp3(sous_titre, audio, titre_modifiable=titre_arabe, chemin_texte_transcris = texte_transcris)

    except IndexError as e:
        print(f"Erreur : Aucun fichier valide trouvé dans le sous-dossier '{sous_dossier}'. Détails : {e}")
    except Exception as e:
        print(f"Erreur inattendue lors du traitement du sous-dossier '{sous_dossier}'. Détails : {e}")

In [ ]:
liste_dossiers_videos=lister_sous_dossiers_avec_videos(chemin_dossier_3)
for dossier in liste_dossiers_videos :
    if recuperer_fichiers_par_suffixe(dossier, extension_revise) != []:
        deplacer_dossier(dossier, chemin_dossier_4)
print("Nombre d'éléments à Vérifier = ", len(lister_sous_dossiers_avec_videos(chemin_dossier_3)))
print("Nombre d'éléments à monter = ", len(lister_sous_dossiers_avec_videos(chemin_dossier_4)))

###Montage

####Testing

In [ ]:
# liste_sous_dossiers = lister_sous_dossiers_avec_videos(chemin_dossier_4)

# # Vérification si des sous-dossiers existent
# if not liste_sous_dossiers:
#     print("Aucun sous-dossier trouvé.")
# else:
#     for sous_dossier in liste_sous_dossiers:
#         print(f"\n--- Traitement du sous-dossier : {sous_dossier} ---")

#         # Localiser les fichiers nécessaires avec gestion d'erreurs
#         try:
#             chemin_video = lister_elements_videos_dossier(sous_dossier)[0]
#             print(f"Vidéo trouvée : {chemin_video}")
#         except IndexError:
#             print("Aucune vidéo trouvée dans ce sous-dossier.")

#         try:
#             sous_titre_revise = recuperer_fichiers_par_suffixe(sous_dossier, extension_revise)[0]
#             print(f"Sous-titre révisé trouvé : {sous_titre_revise}")
#         except IndexError:
#             print("Aucun fichier de sous-titre révisé trouvé.")

#         try:
#             audio = trouver_audio_dans_dossier(sous_dossier)
#             print(f"Audio trouvé : {audio}")
#         except Exception:
#             print("Aucun fichier audio trouvé.")

#         try:
#             titre_path = recuperer_fichiers_par_suffixe(sous_dossier, extension_titre)[0]
#             titre_concret, extension = extract_title_from_path(titre_path)
#             titre_final = titre_concret + extension
#             titre_final = titre_final.replace(extension_titre, "")
#             print(f"Titre extrait : {titre_final}")
#         except IndexError:
#             print("Aucun fichier de titre trouvé.")
#         except Exception as e:
#             print(f"Erreur lors de l'extraction du titre : {e}")

#         # Ajout d'une séparation entre les sous-dossiers
#         print("\n" + "="*50 + "\n")


####Montage

In [ ]:
liste_sous_dossiers =lister_sous_dossiers_avec_videos(chemin_dossier_4)
# liste_sous_dossiers = ["/content/drive/Othercomputers/Mon ordinateur portable/Projet concret (Drive)/4.Sous-titres Vérifié/السنة في إفطار #رمضان # #shorts"]
for sous_dossier in liste_sous_dossiers:
    print(f"\n--- Traitement du sous-dossier : {sous_dossier} ---")

    # Localiser les fichiers nécessaires avec gestion d'erreurs
    try:
        chemin_video = lister_elements_videos_dossier(sous_dossier)[0]
    except IndexError:
        print("Aucune vidéo trouvée dans ce sous-dossier.")
        continue  # Passer au sous-dossier suivant si aucune vidéo

    try:
        sous_titre_revise = recuperer_fichiers_par_suffixe(sous_dossier, extension_revise)[0]
    except IndexError:
        print("Aucun fichier de sous-titre révisé trouvé.")
        continue

    try:
        audio = trouver_audio_dans_dossier(sous_dossier)
    except Exception:
        print("Aucun fichier audio trouvé.")
        continue

    try:
        titre_path = recuperer_fichiers_par_suffixe(sous_dossier, extension_titre)[0]
        titre_concret, extension = extract_title_from_path(titre_path)
        titre_final = titre_concret + extension
        titre_final = titre_final.replace(extension_titre, "")
        print(f"Titre extrait : {titre_final}")
    except IndexError:
        print("Aucun fichier de titre trouvé.")
        continue
    except Exception as e:
        print(f"Erreur lors de l'extraction du titre : {e}")
        continue

    try:
        # Étape 1: Ajouter un rectangle noir avec dégradé sur la moitié inférieure de la vidéo
        video_avec_rectangle = ajouter_rectangle_noir_video(chemin_video, titre_final)  # Appel de la fonction

        # Étape 2: Ajouter un logo sur la vidéo
        video_avec_logo = ajouter_logo_video(video_avec_rectangle)

        # Étape 3: Ajouter les sous-titres sur la vidéo
        video_avec_texte = ajouter_texte_video_opencv(video_avec_logo, sous_titre_revise)

        # Étape 4: Ajouter le titre à la fin de la vidéo
        video_avec_titre = ajouter_titre_debut_video(video_avec_texte, titre_final)

        # Fusionner audio et vidéo
        fusion_result = titre_path.replace(extension_titre, extension_montee)
        print(f"Fusion audio/vidéo en cours : vidéo={video_avec_texte}, audio={audio}, export={fusion_result}")
        fusionner_audio_video(video_avec_texte, audio, fusion_result)
        print(f"Fusion terminée avec succès : {fusion_result}")

        # Suppression des fichiers temporaires
        for fichier in [video_avec_rectangle, video_avec_titre, video_avec_logo, video_avec_texte]:
            if os.path.exists(fichier):
                os.remove(fichier)
                print(f"Fichier temporaire supprimé : {fichier}")
            else:
                print(f"Fichier temporaire introuvable pour suppression : {fichier}")

        print(f"Traitement terminé pour : {sous_dossier}")

    except FileNotFoundError as e:
        print(f"Erreur : {e}")
    except IndexError as e:
        print(f"Erreur : Une liste utilisée dans le sous-dossier '{sous_dossier}' est vide. Détails : {e}")
    except Exception as e:
        print(f"Erreur inattendue lors du traitement du sous-dossier '{sous_dossier}'. Détails : {e}")


In [ ]:
liste_dossiers_videos = lister_sous_dossiers_avec_videos(chemin_dossier_4)

for dossier in liste_dossiers_videos:
    fichiers = recuperer_fichiers_par_suffixe(dossier, extension_montee)
    if fichiers:
        fichier_video = fichiers[0]
        output_path = os.path.join(chemin_dossier_5, "Videos finies")
        deplacer_video(fichier_video, output_path)
        deplacer_dossier(dossier, os.path.join(chemin_dossier_5, "Dossiers_traitement"))


####Maintenant déplacer vers le dossier à poster :-)


##Maintenant déplacer vers le dossier à poster :-)


##Génération de fichiers CSV pour upload en masse sur ZOHO

#Tests

In [ ]:
from bidi.algorithm import get_display

def apply_bidi_correctly(texte):
    """
    Applique l'algorithme BiDi uniquement aux segments arabes du texte.
    """
    segments = []
    current_segment = ""
    current_script = None  # 'latin', 'arabic', or None

    for char in texte:
        # Détecte si le caractère appartient à l'alphabet arabe
        if "\u0600" <= char <= "\u06FF" or "\u0750" <= char <= "\u077F" or "\u08A0" <= char <= "\u08FF":
            script = "arabic"
        else:
            script = "latin"

        # Si le script change, ajouter le segment actuel à la liste
        if script != current_script:
            if current_segment:
                segments.append((current_script, current_segment))
            current_segment = char
            current_script = script
        else:
            current_segment += char

    # Ajouter le dernier segment
    if current_segment:
        segments.append((current_script, current_segment))

    # Appliquer BiDi uniquement aux segments arabes
    processed_segments = []
    for script, segment in segments:
        if script == "arabic":
            # Appliquer BiDi et corriger la direction pour les caractères arabes
            processed_segments.append(get_display(segment[::-1]))  # L'inversion corrige l'ordre
        else:
            processed_segments.append(segment)  # Laisser le texte latin inchangé

    # Recombiner les segments
    return "".join(processed_segments)




Original text: Bonjour مرحبا  bonjour بالعالم Jojo
Processed text (BiDi corrigé): Bonjour مرحبا  bonjour بالعالم Jojo


#Vérifications

In [ ]:
import os

chemin_dossiers = [
    chemin_dossier_0,
    chemin_dossier_1,
    chemin_dossier_2,
    chemin_dossier_3,
    chemin_dossier_4,
    r"/content/drive/MyDrive/Projet concret (Drive)/5.Vidéos Montées/Videos finies",  # Corrigé ici
]

for i, chemin in enumerate(chemin_dossiers):
    try:
        nombre_elements = len(os.listdir(chemin))
        nom_dossier = os.path.basename(chemin)
        print(f"{nom_dossier} = {nombre_elements} éléments")
    except FileNotFoundError:
        print(f"{nom_dossier} = Erreur : dossier introuvable")
    except PermissionError:
        print(f"{nom_dossier} = Erreur : permission refusée")


0.Vidéos sans sous dossier = 1 éléments
1.Vidéos vierges avec dossier = 0 éléments
2.Sous-titres Extrait = 3 éléments
3.Sous-titres Traduit (modèle) = 0 éléments
4.Sous-titres Vérifié = 7 éléments
Videos finies = 14 éléments
